In [1]:
import pandas as pd
import numpy as np

# 数据读取

In [2]:
datas=pd.read_csv('ISEAR.csv',header=None)

In [3]:
datas.head()

,0,1,2
0,joy,On days when I feel close to my partner and ot...,NaN
1,fear,Every time I imagine that someone I love or I ...,NaN
2,anger,When I had been obviously unjustly treated and...,NaN
3,sadness,When I think about the short time that we live...,NaN
4,disgust,At a gathering I found myself involuntarily si...,NaN


## 拆分训练集测试集

In [4]:
from sklearn.model_selection import train_test_split
x=datas[1].values.tolist()
y=datas[0].values.tolist()

x_t,x_v,y_t,y_v=train_test_split(x,y,test_size=0.2,random_state=42)

## 转换词向量

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer()

x_t=vec.fit_transform(x_t)
x_v=vec.transform(x_v)

In [6]:
x_t

<6012x8019 sparse matrix of type '<class 'numpy.float64'>'
	with 105216 stored elements in Compressed Sparse Row format>

In [7]:
x_v

<1504x8019 sparse matrix of type '<class 'numpy.float64'>'
	with 25350 stored elements in Compressed Sparse Row format>

# 构建模型

## 逻辑回归

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV#交叉验证

param={'C':[0.00001,0.0001,0.01,0.005,0.1,0.5,1,2,5,10]}#正则化的强度

lr=LogisticRegression()
lr.fit(x_t,y_t).score(x_v,y_v)

clf=GridSearchCV(lr,param,cv=5)
clf.fit(x_t,y_t)
clf.score(x_v,y_v)
print(clf.best_params_)

C:\Users\Wang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Wang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Wang\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Wang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Wang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.p

{'C': 5}


### 混淆矩阵

In [9]:
from sklearn.metrics import confusion_matrix
cmat=confusion_matrix(y_v,clf.predict(x_v)) #矩阵A[i,j]表示类别i被分类成类别j的个数
print(cmat)

[[101  35  17  26  11  18  19]
 [ 23 122  14  18   8   5  14]
 [ 12   7 141  12   8  11   9]
 [ 25  10  11 110  12  17  24]
 [  5   9   8   9 180  13   9]
 [ 20  16  12   6  18 126   7]
 [ 23  23  17  33  18   8 104]]


In [10]:
right_cnt=0
for i in range(cmat.shape[0]):
    right_cnt+=cmat[i,i]

In [11]:
right_rate=float(right_cnt)/cmat.sum()
print(right_rate)

0.5877659574468085


## 神经网络

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [13]:
#one-hot 编码
m={
    'anger':0,
    'disgust':1,
    'fear':2,
    'guilt':3,
    'guit':4,
    'joy':5,
    'sadness':6,
    'shame':7
}

In [24]:
y_t_hot=[]
for i in y_t:
    t=np.zeros(8)
    t[m[i]]=1
    y_t_hot.append(list(t))
y_t_hot=np.array(y_t_hot)

In [25]:
y_v_hot=[]
for i in y_v:
    t=np.zeros(8)
    t[m[i]]=1
    y_v_hot.append(list(t))
y_v_hot=np.array(y_v_hot)

In [21]:
np.array(y_t_hot)

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [17]:
model=Sequential()
model.add(Dense(1024,input_shape=(8019,),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8,activation='softmax'))

W1015 15:49:16.586622  7396 deprecation_wrapper.py:119] From C:\Users\Wang\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1015 15:49:16.609635  7396 deprecation_wrapper.py:119] From C:\Users\Wang\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1015 15:49:16.614639  7396 deprecation_wrapper.py:119] From C:\Users\Wang\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1015 15:49:16.641658  7396 deprecation_wrapper.py:119] From C:\Users\Wang\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1015 15:49:16.651670  7396 deprecation.py:506] Fr

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              8212480   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 264       
Total para

In [19]:
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.2),metrics=['accuracy'])

W1015 15:49:16.873822  7396 deprecation_wrapper.py:119] From C:\Users\Wang\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1015 15:49:16.884836  7396 deprecation_wrapper.py:119] From C:\Users\Wang\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [26]:
model.fit(x_t,y_t_hot,epochs=20,validation_data=(x_v,y_v_hot))

W1015 15:52:49.718073  7396 deprecation.py:323] From C:\Users\Wang\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 6012 samples, validate on 1504 samples
Epoch 1/20
6012/6012 [==============================] - 19s 3ms/step - loss: 1.9788 - acc: 0.1475 - val_loss: 1.9535 - val_acc: 0.1390
Epoch 2/20
6012/6012 [==============================] - 27s 4ms/step - loss: 1.9483 - acc: 0.1658 - val_loss: 1.9365 - val_acc: 0.1370
Epoch 3/20
6012/6012 [==============================] - 34s 6ms/step - loss: 1.8886 - acc: 0.2359 - val_loss: 1.8856 - val_acc: 0.2188
Epoch 4/20
6012/6012 [==============================] - 36s 6ms/step - loss: 1.7536 - acc: 0.3160 - val_loss: 1.5865 - val_acc: 0.4109
Epoch 5/20
6012/6012 [==============================] - 23s 4ms/step - loss: 1.6182 - acc: 0.3920 - val_loss: 1.6663 - val_acc: 0.4036
Epoch 6/20
6012/6012 [==============================] - 21s 3ms/step - loss: 1.4980 - acc: 0.4513 - val_loss: 1.4351 - val_acc: 0.4887
Epoch 7/20
6012/6012 [==============================] - 20s 3ms/step - loss: 1.3867 - acc: 0.5055 - val_loss: 1.3355 - val_acc: 0.5246
Epoch 8

In [27]:
model.fit(x_t,y_t_hot,epochs=20,validation_data=(x_v,y_v_hot))

Train on 6012 samples, validate on 1504 samples
Epoch 1/20
6012/6012 [==============================] - 20s 3ms/step - loss: 0.8270 - acc: 0.7347 - val_loss: 1.4062 - val_acc: 0.5406
Epoch 2/20
6012/6012 [==============================] - 22s 4ms/step - loss: 0.8379 - acc: 0.7247 - val_loss: 1.5674 - val_acc: 0.5299
Epoch 3/20
6012/6012 [==============================] - 37s 6ms/step - loss: 0.7598 - acc: 0.7595 - val_loss: 1.4362 - val_acc: 0.4787
Epoch 4/20
6012/6012 [==============================] - 27s 4ms/step - loss: 0.6637 - acc: 0.7949 - val_loss: 1.5548 - val_acc: 0.5366
Epoch 5/20
6012/6012 [==============================] - 19s 3ms/step - loss: 0.6319 - acc: 0.8051 - val_loss: 1.3623 - val_acc: 0.5366
Epoch 6/20
6012/6012 [==============================] - 31s 5ms/step - loss: 0.5078 - acc: 0.8426 - val_loss: 1.5551 - val_acc: 0.5632
Epoch 7/20
6012/6012 [==============================] - 25s 4ms/step - loss: 0.5864 - acc: 0.8182 - val_loss: 1.6728 - val_acc: 0.5346
Epoch 8

In [28]:
model.evaluate(x_v,y_v_hot)

1504/1504 [==============================] - 1s 577us/step


[1.561068796096964, 0.5152925531914894]